In [1]:
import os
path_to_spark = "/usr/hdp/2.3.4.0-3485/spark/"
os.environ['SPARK_HOME'] = path_to_spark
import sys
path_to_pyspark = os.path.join(path_to_spark, "python")
sys.path.insert(0, path_to_pyspark)
path_to_py4j ="/usr/hdp/2.3.4.0-3485/spark/python/lib/py4j-0.8.2.1-src.zip"
sys.path.insert(0, path_to_py4j)
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import *
import BeautifulSoup as bs4
from pyspark import SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import pyspark.sql.functions as func
from math import sqrt

In [2]:
#將1.txt~100.txt讀出並接成一個大list

test = []
x = 1
while x < 11 :

    try:        
        f = open('/root/test/'+str(x)+'.txt', 'r')
        f=f.read()
        test.append(f)
        x = x+1
    except IOError:
        print "No such file or directory" + str(x)
        x =x+1
        continue
#print test

In [3]:
sc = SparkContext()

In [4]:
#spark的進入點
test1= sc.parallelize(test)

In [5]:
#將list中的每個值(文本)依據空格將個詞分隔成值在接成一個大list
test2 = test1.flatMap(lambda x: x.split(' '))
test2 = test2.distinct()

In [6]:
#將list中的每個值(文本)依據空格將個詞分隔成值再依據文本分割成不同的list
test1 = test1.map(lambda x: x.split(' '))
test1_collect = test1.collect()
#print test1_collect

In [7]:
#宣告Hash表的大小
hashingTF = HashingTF()

In [8]:
#計算TF值
tf = hashingTF.transform(test1)

In [9]:
#找出每個詞的hash index值
tf1 = test2.map(lambda x: [x,hashingTF.indexOf(x)])
c = tf1.collect()
#print c
#print len(c)

In [10]:
tf.cache()

PythonRDD[7] at RDD at PythonRDD.scala:43

In [11]:
#計算tfidf值
idf = IDF(minDocFreq=5).fit(tf)
tfidf = idf.transform(tf)
d = tfidf.collect()
print d

[SparseVector(1048576, {0: 0.0, 57: 0.9554, 654: 0.0, 920: 0.0, 1346: 0.0, 1530: 0.6369, 1733: 0.0, 1772: 0.0, 2405: 0.452, 2439: 0.0, 2594: 0.0, 2749: 0.0, 2891: 0.0, 3079: 0.0, 3636: 0.0, 3681: 0.0, 3709: 0.0, 3919: 0.0, 4078: 0.0, 4470: 0.0, 4671: 0.452, 5125: 3.2405, 5220: 0.0, 5527: 0.8027, 5607: 0.6061, 5692: 0.6061, 5702: 0.0, 5853: 0.0, 5930: 0.0, 6022: 0.0, 6073: 0.0, 6124: 0.0, 6324: 0.0, 6767: 0.0, 6836: 0.0, 6868: 0.0, 7218: 0.3185, 7344: 0.4013, 7689: 0.0, 8068: 0.0, 8213: 0.0, 8277: 0.0, 8282: 0.2007, 8287: 0.0, 8290: 0.2007, 8952: 1.2123, 8966: 0.0, 8989: 0.904, 9001: 0.0, 9051: 0.0, 9184: 0.0, 9186: 3.0101, 9511: 0.6061, 9518: 0.8027, 9543: 0.0, 9599: 0.6061, 9834: 0.0, 9992: 0.0953, 10131: 0.6061, 10164: 0.0, 10513: 0.0, 10558: 0.0, 10706: 0.0, 10798: 0.0, 11222: 0.0, 11655: 0.0, 11943: 0.0, 12370: 0.0, 12886: 0.6061, 12946: 0.0, 13642: 0.0, 13801: 0.452, 14100: 0.0, 14248: 0.0, 14329: 0.0, 15240: 0.0, 15451: 0.0, 15699: 0.0, 15711: 0.904, 15876: 0.602, 15926: 0.0, 166

In [12]:
#取出n篇文章(n個list)的index表
tfidf_index = tfidf.take(10)

In [13]:
#用全部詞的大list放到第一張index表(tfidf_index[0])中找出值
tmp1 = c
y = 0
while y < len(c) :
    try:        
        tmp1[y].append(tfidf_index[7][tmp1[y][1]])
        y = y+1
    except IndexError:
        print "list index out of range" + str(y)
        y =y+1
        continue

In [14]:
#去掉0的值就得出第一篇文章有哪些詞
word_1= sc.parallelize(tmp1)
bb  = word_1.filter(lambda x: x[2] != 0.0 )
bbb =bb.filter(lambda x : x[2] > 2)
bbbb = bbb.collect()
print bbbb
for i in bbbb :
    x = i[2]
    fid = open('/root/test/0804.txt', 'a') 
    fid.write(str(x)+' '+ str(i[0])+'\n')
    fid.close()

[['\xe6\x88\xb0\xe5\x8a\x9b', 426552, 2.424543214281262], ['\xe6\x83\xa1\xe4\xba\xba', 345912, 2.2599256187152861], ['\xe5\xb0\x8d\xe6\x88\xb0', 264296, 3.0306790178515772], ['\xe9\x9a\x8a\xe5\x93\xa1', 885308, 7.8797654464141011], ['\xe6\x95\x91\xe5\x87\xba', 626344, 2.424543214281262], ['\xe8\xb7\xaf\xe4\xba\xba', 529192, 4.5198512374305722], ['\xe9\x9a\x8a\xe9\x95\xb7', 498456, 3.6217868325643474], ['\xe8\xa6\xaa\xe6\x83\x85', 651552, 3.6158809899444577], ['\xe8\xb7\xaf\xe7\xb7\x9a', 731680, 3.6368148214218929], ['\xe9\x9d\xa9\xe5\x91\xbd', 994076, 2.2599256187152861], ['\xe6\x94\xbb\xe9\x9a\x8a', 849000, 3.6368148214218929], ['\xe8\x88\x9e\xe5\x8f\xb0', 867664, 3.6368148214218929], ['\xe5\x93\xa5\xe5\x93\xa5', 736848, 10.910444464265678], ['\xe9\x85\x8d\xe8\xa7\x92', 751380, 2.2291761178297422], ['\xe6\x8f\x90', 845028, 3.6368148214218929], ['\xe7\xb5\x84\xe6\x88\x90', 27508, 2.424543214281262], ['\xe7\x89\xb9\xe8\x89\xb2', 290008, 3.0100604319322688], ['\xe9\x85\x92\xe5\x90\xa7', 

In [15]:
cat /root/test/0804.txt

2.42454321428 戰力
2.25992561872 惡人
3.03067901785 對戰
7.87976544641 隊員
2.42454321428 救出
4.51985123743 路人
3.62178683256 隊長
3.61588098994 親情
3.63681482142 路線
2.25992561872 革命
3.63681482142 攻隊
3.63681482142 舞台
10.9104444643 哥哥
2.22917611783 配角
3.63681482142 提
2.42454321428 組成
3.01006043193 特色
4.51985123743 酒吧
2.20737765008 流暢
2.42454321428 好多
3.5029910423 友情
2.00670695462 感人
8.27979700908 魔
23.0331605357 死射
3.63681482142 監獄
2.25992561872 男友
4.24295062499 作戰
2.25992561872 傑克
2.42454321428 團結
3.61588098994 復
2.86608358007 高潮
9.09203705355 哈莉
2.54762984895 不得不
20.3963784781 小丑
2.42454321428 硬要
2.42454321428 找出
3.82144477342 無聊
10.6355468595 DC
3.03067901785 一角
2.86608358007 接下來
5.87580660866 漫威
2.42454321428 和解
2.25992561872 抱歉
2.42454321428 正常
3.03067901785 羅比
3.03067901785 大樓
4.84908642856 心臟
3.03067901785 閃電俠
2.42454321428 集合
2.25992561872 偉大
2.42454321428 怎麼樣
4.06786611369 劇照
2.42454321428 還算
3.63681482142 支線
3.41140182286 歡樂
4.24295062499 瑪格
6.36907462237 史密斯
4.84908642856 魔法
2.42454321428

In [16]:
rm /root/test/0804.txt